# Efficient yelp API Core

* Jacob Wang
* 12/20/2022

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

### Load Credentials and Create Yelp API Object

In [2]:
# Load API Credentials
with open('/Users/jacobwang/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

### Define Search Terms and File Paths

In [4]:
# set our API call parameters and filename before the first call
location = 'Philadelphia, PA'
term = 'italian'

In [5]:
## Specify fodler for saving data
FOLDER = 'Data/'
os.makedirs(FOLDER, exist_ok=True)


In [6]:
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [7]:
JSON_FILE

'Data/Philadelphia-italian.json'

### Check if Json File exists and Create it if it doesn't

In [11]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(FOLDER, exist_ok=True)

        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w')as f:
        json.dump([], f)
        
## If it exists, inform user
else: 
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Philadelphia-italian.json already exists.


In [9]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=location,
                                       term=term)
print((results))
type(results)

{'businesses': [{'id': '-cEFKAznWmI0cledNOIQ7w', 'alias': 'gran-caffe-l-aquila-philadelphia', 'name': "Gran Caffe L'Aquila", 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/xKGtEMHg8-r_8XyaC585DQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/gran-caffe-l-aquila-philadelphia?adjust_creative=EPlkYaSpsl5pSK06lCD3BQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=EPlkYaSpsl5pSK06lCD3BQ', 'review_count': 1290, 'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'}, {'alias': 'italian', 'title': 'Italian'}, {'alias': 'gelato', 'title': 'Gelato'}], 'rating': 4.5, 'coordinates': {'latitude': 39.95127, 'longitude': -75.1695}, 'transactions': ['pickup', 'delivery'], 'price': '$$', 'location': {'address1': '1716 Chestnut St', 'address2': '', 'address3': '', 'city': 'Philadelphia', 'zip_code': '19103', 'country': 'US', 'state': 'PA', 'display_address': ['1716 Chestnut St', 'Philadelphia, PA 19103']}, 'phone': '+12155685600', 'display_phone': '(215) 5

dict

In [14]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [21]:
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,-cEFKAznWmI0cledNOIQ7w,gran-caffe-l-aquila-philadelphia,Gran Caffe L'Aquila,https://s3-media3.fl.yelpcdn.com/bphoto/xKGtEM...,False,https://www.yelp.com/biz/gran-caffe-l-aquila-p...,1290,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 39.95127, 'longitude': -75.1695}","[delivery, pickup]",$$,"{'address1': '1716 Chestnut St', 'address2': '...",+12155685600,(215) 568-5600,313.234911
1,6ajnOk0GcY9xbb5Ocaw8Gw,barbuzzo-philadelphia,Barbuzzo,https://s3-media3.fl.yelpcdn.com/bphoto/vCTPQ2...,False,https://www.yelp.com/biz/barbuzzo-philadelphia...,3081,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.5,"{'latitude': 39.94999, 'longitude': -75.16216}","[delivery, pickup]",$$,"{'address1': '110 S 13th St', 'address2': '', ...",+12155469300,(215) 546-9300,877.931790
2,lKU_kljhbEQKIPaM1Zbqiw,little-nonnas-philadelphia,Little Nonna's,https://s3-media2.fl.yelpcdn.com/bphoto/0dQJ4d...,False,https://www.yelp.com/biz/little-nonnas-philade...,1251,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 39.9477864380196, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '1234 Locust St', 'address2': '',...",+12155462100,(215) 546-2100,1036.521920
3,dYinIkKBspHV5hSaukklFg,l-angolo-ristorante-philadelphia-3,L'Angolo Ristorante,https://s3-media1.fl.yelpcdn.com/bphoto/_-xeA0...,False,https://www.yelp.com/biz/l-angolo-ristorante-p...,409,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 39.9196243286133, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '1415 W Porter St', 'address2': N...",+12153894252,(215) 389-4252,3797.786432
4,-bV-HbEV9F0qXvmukmQwbw,luna-byob-philadelphia,Luna BYOB,https://s3-media4.fl.yelpcdn.com/bphoto/kR-O3p...,False,https://www.yelp.com/biz/luna-byob-philadelphi...,6,"[{'alias': 'italian', 'title': 'Italian'}]",5.0,"{'latitude': 39.949678356249585, 'longitude': ...",[],NaN,"{'address1': '227 S 20th St', 'address2': '', ...",+12156932220,(215) 693-2220,507.314834
5,eLbKuFpCtHEjCkoWkDDk7g,trattoria-carina-philadelphia-2,Trattoria Carina,https://s3-media4.fl.yelpcdn.com/bphoto/Q3N-1y...,False,https://www.yelp.com/biz/trattoria-carina-phil...,280,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 39.94878, 'longitude': -75.17803}",[delivery],$$,"{'address1': '2201 Spruce St', 'address2': '',...",+12157325818,(215) 732-5818,798.325153
6,qnprJxrjwuEivitA8V2aEQ,fiorella-philadelphia-2,Fiorella,https://s3-media3.fl.yelpcdn.com/bphoto/ntu7O8...,False,https://www.yelp.com/biz/fiorella-philadelphia...,205,"[{'alias': 'pastashops', 'title': 'Pasta Shops...",4.5,"{'latitude': 39.9388677767588, 'longitude': -7...",[delivery],$$$,"{'address1': '817 Christian St', 'address2': '...",+12153059222,(215) 305-9222,2056.445857
7,xdc0TbKkHMZN3N82oeq2XA,via-locusta-philadelphia,Via Locusta,https://s3-media3.fl.yelpcdn.com/bphoto/B9sNFi...,False,https://www.yelp.com/biz/via-locusta-philadelp...,230,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 39.9490553, 'longitude': -75.1704...",[delivery],$$$,"{'address1': '1723 Locust St', 'address2': '',...",+12156420020,(215) 642-0020,540.974464
8,sVZb87xjhbCWHQ2UXOGLrA,giorgio-on-pine-philadelphia,Giorgio On Pine,https://s3-media1.fl.yelpcdn.com/bphoto/EwboUh...,False,https://www.yelp.com/biz/giorgio-on-pine-phila...,1040,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 39.945339381185, 'longitude': -75...","[delivery, pickup]",$$,"{'address1': '1328 Pine St', 'address2': '', '...",+12155456265,(215) 545-6265,1101.049702
9,2qVZiMB22YPC2HCM62Gnbw,scarpetta-philadelphia,Scarpetta,https://s3-media2.fl.yelpcdn.com/bphoto/G5lGck...,False,https://www.yelp.com/biz/scarpetta-philadelphi...,235,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 39.9498657276508, 'longitude': -7...","[delivery, pickup]",$$$,"{'address1': '210 W Rittenhouse Squa

- Calculate how many pages of results needed to cover the total_results

In [17]:
# Use math.ceil to round up for the total number of pages of results.
import time, math
n_pages = math.ceil((results['total'])/results_per_page)
n_pages

160

In [19]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)


  0%|          | 0/160 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


## Open the Final JSON File with Pandas

In [20]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,wUnLSg_GKfEIQ5CQQ770_g,vetri-cucina-philadelphia,Vetri Cucina,https://s3-media4.fl.yelpcdn.com/bphoto/9c6Xv1...,False,https://www.yelp.com/biz/vetri-cucina-philadel...,521,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 39.94659, 'longitude': -75.16319}",[],$$$$,"{'address1': '1312 Spruce St', 'address2': '',...",12157323478,(215) 732-3478,1047.368173
1,agTcS5LYZJe4gNiXK6nNkw,buca-d-oro-philadelphia,Buca D’Oro,https://s3-media2.fl.yelpcdn.com/bphoto/l5SGpf...,False,https://www.yelp.com/biz/buca-d-oro-philadelph...,24,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 39.947077207787196, 'longitude': ...",[],NaN,"{'address1': '711 Locust St', 'address2': '', ...",12159280556,(215) 928-0556,1632.539850
2,Wtr51cNrv1pYyIRg_IawYQ,the-victor-cafe-philadelphia,The Victor Cafe,https://s3-media3.fl.yelpcdn.com/bphoto/8YUxbV...,False,https://www.yelp.com/biz/the-victor-cafe-phila...,481,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 39.931653, 'longitude': -75.166327}",[],$$,"{'address1': '1303 Dickinson St', 'address2': ...",12154683040,(215) 468-3040,2495.514003
3,Vnagnur1os0ZduOpvDH27A,villa-di-roma-philadelphia,Villa Di Roma,https://s3-media4.fl.yelpcdn.com/bphoto/0y9xMA...,False,https://www.yelp.com/biz/villa-di-roma-philade...,460,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 39.9379266053438, 'longitude': -7...",[delivery],$$,"{'address1': '936 S 9th St', 'address2': None,...",12155921295,(215) 592-1295,2076.116949
4,-Ti5pwj6mA99khsxxur8aQ,ambrosia-ristorante-byob-philadelphia,Ambrosia Ristorante BYOB,https://s3-media1.fl.yelpcdn.com/bphoto/9q5lc9...,False,https://www.yelp.com/biz/ambrosia-ristorante-b...,189,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 39.94997, 'longitude': -75.1796}",[delivery],NaN,"{'address1': '231 S 24th St', 'address2': '', ...",12157032010,(215) 703-2010,835.168898


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
3195,Mt6XTQfwk2DUD3AK29YIBg,melograno-philadelphia,Melograno,https://s3-media1.fl.yelpcdn.com/bphoto/3Zc5Nz...,False,https://www.yelp.com/biz/melograno-philadelphi...,476,"[{'alias': 'italian', 'title': 'Italian'}]",3.5,"{'latitude': 39.9512145, 'longitude': -75.1741...",[delivery],$$$,"{'address1': '2012 Sansom St', 'address2': Non...",12158758116,(215) 875-8116,375.902198
3196,hUmHwBQtGg0iH-PZakQJDw,wm-mulherins-sons-philadelphia,Wm Mulherin's Sons,https://s3-media1.fl.yelpcdn.com/bphoto/7dFVMi...,False,https://www.yelp.com/biz/wm-mulherins-sons-phi...,665,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 39.9717102050781, 'longitude': -7...",[delivery],$$$,"{'address1': '1355 N Front St', 'address2': No...",12152911355,(215) 291-1355,3670.103044
3197,BWITd9JUXP3gSi1z3Y-z8g,in-riva-philadelphia,In Riva,https://s3-media3.fl.yelpcdn.com/bphoto/Pt4x-P...,False,https://www.yelp.com/biz/in-riva-philadelphia?...,774,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.0079689, 'longitude': -75.1923...",[delivery],$$,"{'address1': '4116 Ridge Ave', 'address2': '',...",12154384848,(215) 438-4848,6280.270874
3198,3EzsIRZG_JTrUT38KAuzeQ,le-onde-ristorante-elkins-park,Le Onde Ristorante,https://s3-media3.fl.yelpcdn.com/bphoto/QcdaAE...,False,https://www.yelp.com/biz/le-onde-ristorante-el...,38,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 40.072034, 'longitude': -75.12752}","[pickup, delivery]",NaN,"{'address1': '7909 Park Ave', 'address2': None...",12152775878,(215) 277-5878,13662.133978
3199,-nIXhxPvxST6bTuj9jo1UQ,damò-pasta-lab-philadelphia,DaMò Pasta Lab,https://s3-media1.fl.yelpcdn.com/bphoto/mzGMlz...,False,https://www.yelp.com/biz/dam%C3%B2-pasta-lab-p...,202,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 39.94740428864801, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '105 S 12th St', 'address2': None...",12676871410,(267) 687-1410,1137.206770


In [21]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Philadelphia-italian.csv.gz'

In [22]:
## Save it as a compressed csv (to save space)
final_df.to_csv(csv_file, compression = 'gzip', index = False)

# compare filesize with os module's `os.path.getsize`

In [23]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 3,052,800 Bytes
CSV.GZ FILE: 21,491 Bytes
the csv.gz is 142.05016053231586 times smaller!
